# Test Improvements - UX & FLA Normalization

This notebook tests the following improvements:
1. FLA cap removal
2. FLA relationship normalization (fixes encoding errors, excludes general/punitive damages)
3. Case summary label removal
4. UX improvements (better sizing, color contrast)

Run these tests before committing the PR to ensure all functionality works correctly.

In [ ]:
import json
import re
from typing import Optional, List, Dict, Any
import pandas as pd

# Import the normalization function
import sys
sys.path.append('app/ui')
from fla_analytics import normalize_fla_relationship, extract_fla_awards

## Test 1: FLA Relationship Normalization

Test that relationships are properly extracted and encoded errors are fixed.

In [ ]:
# Test cases for relationship normalization
test_cases = [
    # (input, expected_output)
    ("€50,000 to spouse", "Spouse"),
    ("Spouse - loss of guidance", "Spouse"),
    ("Wife's claim", "Spouse"),
    ("Husband", "Spouse"),
    ("Child - loss of care", "Child"),
    ("Son's claim", "Child"),
    ("Daughter", "Child"),
    ("Mother's claim", "Parent"),
    ("Father - loss of companionship", "Parent"),
    ("Brother's claim", "Sibling"),
    ("Sister", "Sibling"),
    ("Grandmother's claim", "Grandparent"),
    ("Grandfather", "Grandparent"),
    ("General damages", None),  # Should be excluded
    ("Punitive damages", None),  # Should be excluded
    ("Aggravated damages", None),  # Should be excluded
    ("Special damages", None),  # Should be excluded
    ("Costs", None),  # Should be excluded
    ("â€ spouse claim", "Spouse"),  # Encoding error fix
    ("€ child", "Child"),  # Euro sign removal
]

print("Testing FLA Relationship Normalization")
print("=" * 80)

passed = 0
failed = 0

for input_desc, expected in test_cases:
    result = normalize_fla_relationship(input_desc)
    
    if result == expected:
        status = "✓"
        passed += 1
    else:
        status = "✗"
        failed += 1
    
    print(f"{status} '{input_desc:40}' -> '{result}' (expected: '{expected}')")

print("=" * 80)
print(f"Results: {passed} passed, {failed} failed")

if failed == 0:
    print("\n✅ All FLA normalization tests passed!")
else:
    print(f"\n⚠️ {failed} test(s) failed!")

## Test 2: FLA Awards Extraction

Test that FLA awards are correctly extracted with normalized relationships.

In [ ]:
# Sample case data
sample_case = {
    'case_name': 'Test v. Case',
    'year': 2023,
    'court': 'ONSC',
    'extended_data': {
        'family_law_act_claims': [
            {
                'description': '€50,000 - Spouse loss of guidance',
                'amount': 50000,
                'category': 'FLA'
            },
            {
                'description': 'Child - loss of care and companionship',
                'amount': 30000,
                'category': 'FLA'
            },
            {
                'description': 'General damages',
                'amount': 100000,
                'category': 'General'
            },
            {
                'description': 'Punitive damages',
                'amount': 25000,
                'category': 'Punitive'
            }
        ]
    }
}

awards = extract_fla_awards(sample_case)

print("Testing FLA Awards Extraction")
print("=" * 80)
print(f"Total claims in case: {len(sample_case['extended_data']['family_law_act_claims'])}")
print(f"Valid FLA relationship claims: {len(awards)}")
print()

for i, award in enumerate(awards, 1):
    print(f"Award {i}:")
    print(f"  Normalized: {award['description']}")
    print(f"  Original: {award['original_description']}")
    print(f"  Amount: ${award['amount']:,}")
    print()

print("Expected behavior:")
print("  - Should extract 2 awards (Spouse and Child)")
print("  - Should exclude 'General damages' and 'Punitive damages'")
print("  - Should normalize 'Spouse' and 'Child' relationships")
print("  - Should fix encoding errors (€ symbols removed)")
print()

if len(awards) == 2:
    print("✅ Extraction test passed!")
else:
    print(f"⚠️ Expected 2 awards, got {len(awards)}")

## Test 3: Load Real Data and Check Distribution

Test on actual compendium data to see the impact of normalization.

In [ ]:
# Load actual data
try:
    with open('data/damages_with_embeddings.json', 'r') as f:
        cases = json.load(f)
    print(f"Loaded {len(cases)} cases")
except Exception as e:
    print(f"Error loading data: {e}")
    print("Trying alternative path...")
    try:
        with open('damages_full.json', 'r') as f:
            cases = json.load(f)
        print(f"Loaded {len(cases)} cases from damages_full.json")
    except:
        print("Could not load any data file")
        cases = []

In [ ]:
# Extract all FLA claims
from collections import Counter

all_awards = []
excluded_count = 0
original_descriptions = []
normalized_relationships = []

for case in cases:
    extended_data = case.get('extended_data', {})
    fla_claims = extended_data.get('family_law_act_claims', [])
    
    for claim in fla_claims:
        desc = claim.get('description', '')
        amount = claim.get('amount', 0)
        
        original_descriptions.append(desc)
        
        normalized = normalize_fla_relationship(desc)
        
        if normalized:
            normalized_relationships.append(normalized)
            if amount > 0:
                all_awards.append(amount)
        else:
            excluded_count += 1

print("FLA Claims Analysis")
print("=" * 80)
print(f"Total FLA claims found: {len(original_descriptions)}")
print(f"Valid relationship claims: {len(normalized_relationships)}")
print(f"Excluded (general/punitive/costs): {excluded_count}")
print()

if normalized_relationships:
    relationship_counts = Counter(normalized_relationships)
    
    print("Normalized Relationship Distribution:")
    for relationship, count in relationship_counts.most_common():
        print(f"  {relationship:20} : {count:4} ({count/len(normalized_relationships)*100:.1f}%)")
    print()

if all_awards:
    import numpy as np
    print("Award Statistics (Valid FLA Relationships Only):")
    print(f"  Count: {len(all_awards)}")
    print(f"  Min: ${min(all_awards):,}")
    print(f"  Median: ${np.median(all_awards):,.0f}")
    print(f"  Max: ${max(all_awards):,}")
    print()

print("✅ Data analysis complete!")

## Test 4: Encoding Error Detection

Find and display cases with encoding errors that are being fixed.

In [ ]:
# Find encoding errors
encoding_errors = []

for desc in original_descriptions:
    if desc and ('€' in desc or 'â' in desc):
        normalized = normalize_fla_relationship(desc)
        encoding_errors.append({
            'original': desc,
            'normalized': normalized
        })

print("Encoding Error Fixes")
print("=" * 80)
print(f"Found {len(encoding_errors)} descriptions with encoding errors")
print()

if encoding_errors:
    print("Sample fixes (first 10):")
    for i, error in enumerate(encoding_errors[:10], 1):
        print(f"{i}. Original:   '{error['original']}'")
        print(f"   Normalized: '{error['normalized']}'")
        print()
    
    print(f"✅ All encoding errors are being fixed!")
else:
    print("No encoding errors found in current dataset")

## Test 5: Verify FLA Cap Removal

Verify that FLA cap references are removed from the analytics module.

In [ ]:
# Check the fla_analytics.py file for cap references
import os

fla_file_path = 'app/ui/fla_analytics.py'

if os.path.exists(fla_file_path):
    with open(fla_file_path, 'r') as f:
        content = f.read()
    
    # Check for cap-related keywords
    cap_keywords = ['FLA_DAMAGES_CAP', 'fla_cap', 'create_fla_cap_chart']
    found_caps = []
    
    for keyword in cap_keywords:
        if keyword in content:
            found_caps.append(keyword)
    
    print("FLA Cap Reference Check")
    print("=" * 80)
    
    if found_caps:
        print(f"⚠️ Found {len(found_caps)} cap-related references:")
        for keyword in found_caps:
            print(f"  - {keyword}")
    else:
        print("✅ No FLA cap references found - cap has been successfully removed!")
    
    # Check for new distribution chart
    if 'create_fla_distribution_chart' in content:
        print("✅ New distribution chart function found")
    else:
        print("⚠️ Distribution chart function not found")
else:
    print(f"File not found: {fla_file_path}")

## Test 6: Case Summary Display

Verify that the "Case Summary:" label is removed from search results.

In [ ]:
# Check streamlit_app.py for case summary label
streamlit_file_path = 'streamlit_app.py'

if os.path.exists(streamlit_file_path):
    with open(streamlit_file_path, 'r') as f:
        lines = f.readlines()
    
    # Find case summary related lines
    summary_lines = []
    for i, line in enumerate(lines, 1):
        if 'Case Summary' in line or 'summary_text' in line:
            summary_lines.append((i, line.strip()))
    
    print("Case Summary Display Check")
    print("=" * 80)
    
    has_label = any('"**Case Summary:**"' in line or "'**Case Summary:**'" in line 
                    for _, line in summary_lines)
    
    if has_label:
        print("⚠️ Found 'Case Summary:' label in code")
    else:
        print("✅ 'Case Summary:' label has been removed!")
    
    print(f"\nFound {len(summary_lines)} lines related to summary display:")
    for line_num, line in summary_lines[:5]:  # Show first 5
        print(f"  Line {line_num}: {line[:80]}")
else:
    print(f"File not found: {streamlit_file_path}")

## Test 7: CSS Improvements

Verify that CSS has been updated for better contrast and sizing.

In [ ]:
# Check CSS improvements in streamlit_app.py
if os.path.exists(streamlit_file_path):
    with open(streamlit_file_path, 'r') as f:
        content = f.read()
    
    # Extract CSS block
    import re
    css_match = re.search(r'st\.markdown\("""\s*<style>(.*?)</style>', content, re.DOTALL)
    
    if css_match:
        css_content = css_match.group(1)
        
        print("CSS Improvements Check")
        print("=" * 80)
        
        # Check for improved properties
        improvements = [
            ('Better font sizes', 'font-size: 1.05rem' in css_content or 'font-size: 1.1rem' in css_content),
            ('Line height for readability', 'line-height: 1.6' in css_content or 'line-height: 1.7' in css_content),
            ('Improved color contrast', '#111827' in css_content or '#1f2937' in css_content),
            ('Better padding/spacing', 'padding: 1.25rem' in css_content or 'padding: 1.35rem' in css_content),
            ('Expander improvements', 'expanderHeader' in css_content),
            ('Metric improvements', 'stMetricValue' in css_content),
        ]
        
        passed_improvements = sum(1 for _, check in improvements if check)
        
        for improvement, check in improvements:
            status = "✅" if check else "❌"
            print(f"{status} {improvement}")
        
        print()
        print(f"UX Improvements: {passed_improvements}/{len(improvements)} implemented")
        
        if passed_improvements >= len(improvements) - 1:  # Allow 1 variation
            print("\n✅ CSS improvements successfully applied!")
        else:
            print(f"\n⚠️ Only {passed_improvements} improvements found")
    else:
        print("Could not find CSS block in streamlit_app.py")
else:
    print(f"File not found: {streamlit_file_path}")

## Summary

Run all cells above to verify:

1. ✅ FLA relationship normalization works correctly
2. ✅ Encoding errors (€ symbols) are fixed
3. ✅ General/punitive damages are excluded
4. ✅ FLA cap references are removed
5. ✅ Distribution chart replaces cap chart
6. ✅ "Case Summary:" label is removed
7. ✅ CSS improvements for better UX

All tests should pass before merging the PR!